## Imports



In [ ]:
import numpy as np
import torch

## Data loading

In [ ]:
# Loading of the test dataset of 10 nodes
number_nodes = 10
test_loader_10 = np.loadtxt("tsp-data/tsp10_test_concorde.txt", usecols=np.concatenate((np.arange(0,number_nodes*2),np.arange(number_nodes*2+1,number_nodes*3+2))))

In [ ]:
# Splitting between point coordinates and path
test_input_10 = test_loader_10[:,:number_nodes*2]
test_input_10 = test_input_10.reshape(test_input_10.shape[0], number_nodes, 2)
test_path_10 = test_loader_10[:,number_nodes*2:]

## Initialization of $W$ and $\theta$ 



In [ ]:
def init_weights(nodes,A,B,C,D):
    n, _ = nodes.shape
    distances = np.sqrt(((nodes[None,:,:] - nodes[:,None,:])**2).sum(axis=2))
    weight = np.zeros((n,n,n,n))
    id = np.identity(n)
    non_id = (id==0)
    id_left = np.roll(id, -1)
    id_right = np.roll(id, 1)
    weights = -A*(id[:,:,None,None]*non_id[None,None,:,:]) -B*(non_id[:,:,None,None]*id[None,None,:,:]) - C - D*(id_left[None,None,:,:] + id_right[None,None,:,:])*distances[:,:,None,None]
    weights = np.transpose(weights,(0,2,1,3))
    weights = np.reshape(weights, (n*n,n*n))
    return weights

def init_threshold(C,N):
    return -np.ones(N*N)*C*N

## Model

In [ ]:
def update(N, weights, threshold, iter):
    values = 1/(N)+(np.random.rand(N*N)-0.5)/10000
    nodes = activation(values)
    for i in range(iter):
        s = (weights*nodes).sum(axis=1)
        values = values + (-values + s + threshold)*0.000001
        nodes = activation(values)
    return nodes

In [ ]:
def activation(ar):
    return 0.5*(1+np.tanh(ar/0.02))

In [ ]:
def post_process(values, N):
    values = np.reshape(values, (N,N))
    a = np.zeros(N)
    for i in range(N):
        arg = np.argmax(values)
        a[arg%10]=arg//10
        values[(arg//10),:] = -1
        values[:,(arg%10)] = -1
    return a

## Evaluation functions

In [ ]:
def adjacency(path):
  path = torch.tensor(path)
  heads = path.int()
  tails = torch.roll(heads,-1)
  adjacency_matrix = np.zeros((len(path),len(path)))
  adjacency_matrix[heads,tails] = 1
  adjacency_matrix = torch.tensor(adjacency_matrix, dtype = torch.int64)
  return adjacency_matrix

In [ ]:
def adjacency_matrix(path):
  return(adjacency(path[:-1]-1))

In [ ]:
def dist(nodes, path, target):
  if target:
    adj_matrix = adjacency(path)
  else:
    adj_matrix = adjacency_matrix(path)
  distances = np.sqrt(((nodes[None,:,:] - nodes[:,None,:])**2).sum(axis=2))
  return (adj_matrix * distances).sum(axis=(0,1))

In [ ]:
def iteration(test_input, test_path, nb_nodes, j):
  nodes = test_input[j]
  opt_path = test_path[j]
  values = update(nb_nodes, init_weights(nodes,A,B,C,D), init_threshold(C,nb_nodes), 1000)
  path = post_process(values, nb_nodes)
  hop_dist = dist(nodes, path, True)
  opt_dist = dist(nodes, opt_path, False)
  return hop_dist, opt_dist

## Evaluation

In [ ]:
# Hyperparameters
A = 0.7
B = 0.7
C = 16
D = 8

In [ ]:
hop_distances = []
opt_distances = []
for j in range(len(test_path_10)):
  h, o = iteration(test_input_10, test_path_10, 10, j)
  hop_distances.append(h)
  opt_distances.append(o)
print((np.array(hop_distances)/np.array(opt_distances)).mean()-1)